In [8]:
import sys
import os
!pip install pyyaml
!pip install ensure

# Add the absolute path to your project root (where `src/` lives)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


In [9]:
%pwd


'd:\\MLOps\\DS_MLOPS_Eproject\\research'

In [10]:
os.chdir("../")
%pwd

'd:\\MLOps\\DS_MLOPS_Eproject'

In [11]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [ ]:
# Install all required packages (only run once)
!pip install pyyaml ensure python-box joblib
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Import necessary modules
# from src.datascience.entity.config_entity import DataIngestionConfig  # adjust if it's in another file
from src.datascience.constants import *  # be cautious with wildcard imports in production
from src.datascience.utils.common import read_yaml, create_directories


In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [14]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [15]:
## componants -Data Ingestion
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config
    
    #downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists:")

    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """ 
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config= ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Expection as e:
    raise e

[2025-06-24 16:48:06,491:INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-24 16:48:06,515:INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-24 16:48:06,521:INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-24 16:48:06,529:INFO: common: created directory at: artifacts]
[2025-06-24 16:48:06,533:INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-24 16:48:07,995:INFO: 4250935231: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 53FA:11C2F1:3B0ED0:84EABB:685A896F
Via: 1.1 varnish, 1.1 varnish
Expires: T